# Secure Data Access To S3 Bucket via IAM Policy using VPC

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

s3 = boto3.Session().client(service_name="s3", region_name=region)
iam = boto3.Session().client(service_name="iam", region_name=region)

In [ ]:
role_name = role.split("/")[-1]
print(role_name)

In [ ]:
import time

timestamp = int(time.time())

In [ ]:
bucket_secure_name = "bucket-secure-iam-policy-vpc-{}".format(timestamp)
print(bucket_secure_name)

In [ ]:
!aws s3 mb s3://$bucket_secure_name

In [ ]:
!aws s3 ls s3://$bucket_secure_name

In [ ]:
policy_name = "DSOAWS_Secure_IAMPolicy_VPC_{}".format(timestamp)
print(policy_name)

In [ ]:
different_vpc_id = "blah"

In [ ]:
# Create the IAM policy
iam_policy_deny = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Deny",
            "Action": ["s3:ListBucket"],
            "Resource": ["arn:aws:s3:::{}".format(bucket_secure_name)],
            "Condition": {"StringNotEquals": {"aws:sourceVpc": different_vpc_id}},
        }
    ],
}

In [ ]:
import json

response = iam.put_role_policy(RoleName=role_name, PolicyName=policy_name, PolicyDocument=json.dumps(iam_policy_deny))

print(response)

time.sleep(30)

In [ ]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Policy

In [ ]:
response = iam.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
print(response)

time.sleep(30)

# Verify Access After Removing the Policy

In [ ]:
!aws s3 ls s3://$bucket_secure_name

# Remove the Bucket

In [ ]:
!aws s3 rm s3://$bucket_secure_name

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}